In [1]:
import requests
from requests.exceptions import RequestException
import random

/Users/jonathanbizet/Documents/question_type_examen/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
api_url = "https://jv6gjfb6zh.execute-api.eu-west-3.amazonaws.com/v1"

In [3]:
def get_question_by_theme(theme):
    try:
        response = requests.get(f"{api_url}/theme/{theme}")
        response.raise_for_status()  # Raise an exception for non-2xx status codes
        data = list()
        if theme == "TYPE-EXAMEN":
            for q in response.json():
                if "&&" not in  q["answer"] :
                    data.append(q)
            return data
        return response.json()
    except RequestException as e:
        print(f"An error occurred: {e}")
        return []


In [25]:
question_examen = get_question_by_theme("TYPE-EXAMEN")

In [5]:
def get_event(theme):
    try:
        response = requests.get(f"{api_url}/event", params={'theme': theme})
        response.raise_for_status()  # Raise an exception for non-2xx status codes

        return response.json()
    except RequestException as e:
        print(f"An error occurred: {e}")
        return []

In [10]:
import json
from io import StringIO
sio = StringIO(json.dumps(get_event("TYPE-EXAMEN")))

In [13]:
import pandas as pd
df_event = pd.DataFrame(get_event("TYPE-EXAMEN"))

In [16]:
df_event.columns

Index(['theme', 'id-event', 'event-type', 'id-question', 'timestamp'], dtype='object')

In [44]:
import pandas as pd
df_event = pd.DataFrame(get_event("TYPE-EXAMEN"))
df_event_ko = df_event[df_event['event-type'] == "KO" ]
most_ko_q=df_event_ko.groupby(by='id-question')\
          .count()[["id-event"]]\
          .sort_values(by="id-event", ascending=False)\
          .head(10).index.to_list()

In [28]:
question = list(set(map(lambda x : x["id-question"],question_examen)))

In [45]:
question_nerver_asked = list(filter(lambda x : x not in set(df_event["id-question"]), question))

In [48]:
df_event['timestamp'] = pd.to_datetime(df_event['timestamp'])

In [53]:
df_event.sort_values(by=["timestamp"])['id-question'].to_list()

103    a2d57b16-defc-4493-937f-8caed883dbba
129    679f31d8-821e-4db7-adf1-d6d4773a98e4
139    d1c32481-94ad-46c8-bf29-b2fabccd5065
55     9d5b595c-6818-4f77-9d93-a7776e4d3fb9
82     918daf85-eea9-4229-aad3-3645446d0a36
                       ...                 
114    d521e940-7c11-41b8-9b2c-b230823939b1
68     05dead38-24b7-4ee2-b551-28d37f03b5d8
107    69199ad3-684a-4e9d-b4f2-9b7476866479
25     8894a76a-e69c-4345-8cb5-8958b246fe9a
122    8894a76a-e69c-4345-8cb5-8958b246fe9a
Name: id-question, Length: 152, dtype: object

In [54]:
id_question = "a2d57b16-defc-4493-937f-8caed883dbba"
response = requests.get(f"{api_url}/question/{id_question}")

In [56]:
response.json()

{'theme': 'TYPE-EXAMEN',
 'question': 'A development team runs monthly resource-intensive tests on its general purpose Amazon RDS for MySQL DB instance with Performance Insights enabled. The testing lasts for 48 hours once a month and is the only process that uses the database. The team wants to reduce the cost of running the tests without reducing the compute and memory attributes of the DB instance. Which solution meets these requirements MOST cost-effectively?',
 'options': ['Stop the DB instance when tests are completed. Restart the DB instance when required.',
  'Use an Auto Scaling policy with the DB instance to automatically scale when tests are completed.',
  'Create a snapshot when tests are completed. Terminate the DB instance and restore the snapshot when required.',
  'Modify the DB instance to a low-capacity instance when tests are completed. Modify the DB instance again when required.'],
 'img_path': None,
 'answer': 'Create a snapshot when tests are completed. Terminate 

In [57]:
import pandas as pd
selected_id_question = list()
df_event = pd.DataFrame(get_event("TYPE-EXAMEN"))
df_event_ko = df_event[df_event['event-type'] == "KO"]
most_ko_question = df_event_ko.groupby(by='id-question')\
    .count()[["id-event"]]\
    .sort_values(by="id-event", ascending=False)\
    .head(10).index.to_list()
response = requests.get(f"{api_url}/theme/TYPE-EXAMEN")
question_examen = response.json()
question = list(set(map(lambda x: x["id-question"], question_examen)))
question_nerver_asked = list(
    filter(lambda x: x not in set(df_event["id-question"]), question))
asked_question = df_event.sort_values(by=["timestamp"])[
    'id-question'].to_list()
selected_id_question += question_nerver_asked
selected_id_question += most_ko_question
if len(selected_id_question) > 65:
    selected_id_question = selected_id_question[:65]
elif len(selected_id_question) < 65:
    selected_id_question += asked_question[:65-len(selected_id_question)]

In [60]:
list(filter(lambda x : x['id-question'] in selected_id_question,  question_examen))

[{'theme': 'TYPE-EXAMEN',
  'question': 'A company runs its infrastructure on AWS and has a registered base of 700,000 users for its document management application. The company intends to create a product that converts large .pdf files to .jpg image files. The .pdf files average 5 MB in size. The company needs to store the original files and the converted files. A solutions architect must design a scalable solution to accommodate demand that will grow rapidly over time. Which solution meets these requirements MOST cost-effectively?',
  'options': ['Save the .pdf files to Amazon S3. Configure an S3 PUT event to invoke an AWS Lambda function to convert the files to .jpg format and store them back in Amazon S3.',
   'Save the .pdf files to Amazon DynamoDUse the DynamoDB Streams feature to invoke an AWS Lambda function to convert the files to .jpg format and store them back in DynamoDB.',
   'Upload the .pdf files to an AWS Elastic Beanstalk application that includes Amazon EC2 instances,

In [61]:
list(filter(lambda q: "PROXY" in q["answer"].upper(),question_examen))

[{'theme': 'TYPE-EXAMEN',
  'question': 'A company is running a business-critical web application on Amazon EC2 instances behind an Application Load Balancer. The EC2 instances are in an Auto Scaling group. The application uses an Amazon Aurora PostgreSQL database that is deployed in a single Availability Zone. The company wants the application to be highly available with minimum downtime and minimum loss of data. Which solution will meet these requirements with the LEAST operational effort?',
  'options': ['Place the EC2 instances in different AWS Regions. Use Amazon Route 53 health checks to redirect traffic. Use Aurora PostgreSQL Cross-Region Replication.',
   'Configure the Auto Scaling group to use multiple Availability Zones. Configure the database as Multi-AZ. Configure an Amazon RDS Proxy instance for the database.',
   'Configure the Auto Scaling group to use one Availability Zone. Generate hourly snapshots of the database. Recover the database from the snapshots in the event 